### importing required libraries Note:We will import sklearn when it will be required 


In [1]:
import numpy as np #library to do some complex Mathematics
import pandas as pd #library for data Analysis
import json #library to handle json files
import requests #for knocking api servers
from geopy.geocoders import Nominatim# to grab some Locations
from pandas.io.json import json_normalize #transfporm json file to pandas Data frame
#matplotlib modules for visualizing
import matplotlib.cm as cm
import matplotlib.colors as colors 
import folium #map rendering

## 1.  Explore Dataset

### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

### Just make sure:

##### 1- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
##### 2- to generate maps to visualize your neighborhoods and how they cluster together.

In [2]:
toronto_df = pd.read_csv("data_from_part2.csv",index_col=0)

In [3]:
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286
4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715


In [4]:

toronto_df["Borough"].value_counts()
#we will later explore North York

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           11
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Queen's Park         2
Mississauga          1
Name: Borough, dtype: int64

#### Use geopy library to get the latitude and longitude values of New York City.

In [5]:
address ="Toronto,Ontario"
geolocator = Nominatim(user_agent="a")
location = geolocator.geocode(address)
latitude = location.latitude
longitude= location.longitude
print(f"lat is {latitude},lng is {longitude} of toronto")

lat is 43.653963,lng is -79.387207 of toronto


### Create Folium Map of Toronto

In [6]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=12)

#add markers to map
for lat,lng,borough,neighbourhood in zip(toronto_df["Latitude"],
                                        toronto_df["Longitude"],
                                        toronto_df["Borough"],
                                        toronto_df["Neighbourhood"]):
    label=f"{borough}, {neighbourhood}"
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color="violet",
                       fill=True,fill_color="orange",fill_opacity=0.8,
                       parse_html=False).add_to(map_toronto)
# map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in North York. So let's slice the original dataframe and create a new dataframe of the North York Data.

In [71]:
north_york_df = toronto_df[toronto_df["Borough"]=="North York"].reset_index(drop=True)

In [72]:
north_york_df.drop("Postcode",axis=1,inplace=True)

In [73]:
north_york_df.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.752420,-79.329242
1,North York,Victoria Village,43.730600,-79.313265
2,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286
3,North York,Don Mills North,43.749055,-79.362227
4,North York,Glencairn,43.707990,-79.448367


Let's get the geographical coordinates of North York.

In [74]:
address = 'North York'

geolocator = Nominatim(user_agent="a",timeout=4)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.4491169663959.


Creating Folium Map of North York

In [75]:
map_north_york = folium.Map(location=[latitude,longitude],zoom_start=12)

#add markers to map
for lat,lng,borough,neighbourhood in zip(north_york_df["Latitude"],
                                        north_york_df["Longitude"],
                                        north_york_df["Borough"],
                                        north_york_df["Neighbourhood"]):
    label=f"{borough}, {neighbourhood}"
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color="red",
                       fill=True,fill_color="orange",fill_opacity=0.8,
                       parse_html=False).add_to(map_north_york)
map_north_york

In [76]:
CLIENT_ID = 'YAXPUG5FDSDEY44BIZ0S3WFLS5TB0NEVUMFY2EW0LWGXK2QY'
CLIENT_SECRET = 'AF2D4DEYTAVXRYMRU0TFMVYD1GVHR3U2MBHTGCAXU5SCNNZR' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YAXPUG5FDSDEY44BIZ0S3WFLS5TB0NEVUMFY2EW0LWGXK2QY
CLIENT_SECRET:AF2D4DEYTAVXRYMRU0TFMVYD1GVHR3U2MBHTGCAXU5SCNNZR


#### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters

First, let's create the GET request URL. Name your URL **url**.

In [77]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    north_york_df.loc[0,"Latitude"], 
    north_york_df.loc[0,"Longitude"], 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=YAXPUG5FDSDEY44BIZ0S3WFLS5TB0NEVUMFY2EW0LWGXK2QY&client_secret=AF2D4DEYTAVXRYMRU0TFMVYD1GVHR3U2MBHTGCAXU5SCNNZR&v=20180605&ll=43.75242000000003,-79.32924245299995&radius=500&limit=100'

Send the GET request and examine the resutls

In [78]:
results = requests.get(url).json()

In [79]:
results["response"]["groups"][0]["items"]

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
   'name': 'Brookbanks Park',
   'location': {'address': 'Toronto',
    'lat': 43.751976046055574,
    'lng': -79.33214044722958,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.751976046055574,
      'lng': -79.33214044722958}],
    'distance': 238,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'shortName': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'},
 {'reasons': {'count': 0,
   'items': [{'

In [80]:
def get_category(row):
    try:
        categories_list = row["categories"]
    except:
        categories_list = row["venue.categories"]
        
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]["name"]

In [81]:
venues = results["response"]["groups"][0]["items"]

In [82]:
nearby_venues = json_normalize(venues)

In [83]:
nearby_venues.head() 

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4e8d9dcdd5fbbbb6b3003c7b-0,0,"[{'summary': 'This spot is popular', 'type': '...",4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,Toronto,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",238,CA,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[]
1,e-0-4cb11e2075ebb60cd1c4caad-1,0,"[{'summary': 'This spot is popular', 'type': '...",4cb11e2075ebb60cd1c4caad,Variety Store,29 Valley Woods Road,43.751974,-79.333114,"[{'label': 'display', 'lat': 43.75197441585782...",315,CA,Toronto,ON,Canada,"[29 Valley Woods Road, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",0,[]
2,e-0-53622a89498ed84d6853265e-2,0,"[{'summary': 'This spot is popular', 'type': '...",53622a89498ed84d6853265e,TTC stop - 44 Valley Woods,44 Valley Woods Rd,43.755402,-79.333741,"[{'label': 'display', 'lat': 43.75540238129278...",490,CA,Toronto,ON,Canada,"[44 Valley Woods Rd, Toronto ON, Canada]","[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",0,[]


In [84]:
nearby_venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.cc', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups'],
      dtype='object')

In [85]:
filtered_columns = ["venue.name","venue.categories",'venue.location.lat','venue.location.lng',]

In [86]:
nearby_venues =nearby_venues.loc[:,filtered_columns]

In [87]:
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.332140
1,Variety Store,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",43.751974,-79.333114
2,TTC stop - 44 Valley Woods,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",43.755402,-79.333741


In [88]:
# filter the category for each row
nearby_venues["venue.categories"] = nearby_venues.apply(get_category,axis=1)

#clean column
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [89]:
nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,TTC stop - 44 Valley Woods,Bus Stop,43.755402,-79.333741


In [90]:
print(f"{nearby_venues.shape[0]} venues were returned by foursquare")

3 venues were returned by foursquare


## 2. Explore Neighborhoods in North York

In [91]:
def get_nearby_venues(names,latitude,longitude,radius =500):
    venues_list = []
    for name,lat,lng in zip(names,latitude,longitude):
        
        print(name)
        url=f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={radius}&limit={LIMIT}"

#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        #make the Get Request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        #return only relevant info for each nearby venue
        venues_list.append([(name,lat,lng,
                             v["venue"]["name"],
                             v["venue"]["location"]["lat"],
                            v["venue"]["location"]["lng"],
                            v["venue"]["categories"][0]['name']) for v in results])
    
    nearby_venues_df = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues_df.columns=['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues_df)
        
        
        
        

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *northyork_venues*.

In [92]:
north_york_venues = get_nearby_venues(names=north_york_df["Neighbourhood"],
                                     latitude=north_york_df["Latitude"],
                                      longitude=north_york_df["Longitude"])

Parkwoods
Victoria Village
Lawrence Heights, Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park, Don Mills South
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
CFB Toronto, Downsview East
Silver Hills, York Mills
Downsview West
Downsview, North Park, Upwood Park
Humber Summit
Newtonbrook, Willowdale
Downsview Central
Bedford Park, Lawrence Manor East
Emery, Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [93]:
north_york_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75242,-79.329242,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75242,-79.329242,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.75242,-79.329242,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.73060,-79.313265,Stellar's Lenonade Stand,43.731507,-79.311712,Food Stand
4,Victoria Village,43.73060,-79.313265,Wigmore Park,43.731023,-79.310771,Park


In [94]:
print(north_york_venues.shape[0])

282


Let's check how many venues were returned for each neighborhood

In [95]:
north_york_venues.groupby("Neighbourhood").count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",19,19,19,19,19,19
"CFB Toronto, Downsview East",4,4,4,4,4,4
Don Mills North,5,5,5,5,5,5
Downsview Central,2,2,2,2,2,2
Downsview Northwest,22,22,22,22,22,22
Downsview West,11,11,11,11,11,11
"Downsview, North Park, Upwood Park",3,3,3,3,3,3
"Emery, Humberlea",4,4,4,4,4,4


In [96]:
print(f"there are {north_york_venues['Venue Category'].nunique()} unique categories")

there are 113 unique categories


## 3. Analyze Each Neighborhood

In [97]:
north_york_one_hot =pd.get_dummies(north_york_venues["Venue Category"])
north_york_one_hot.columns

Index(['Airport', 'American Restaurant', 'Arts & Crafts Store',
       'Asian Restaurant', 'Bakery', 'Bank', 'Bar', 'Baseball Field',
       'Basketball Court', 'Beer Store',
       ...
       'Sushi Restaurant', 'Tea Room', 'Thai Restaurant', 'Theater',
       'Toy / Game Store', 'Trail', 'Video Game Store',
       'Vietnamese Restaurant', 'Wings Joint', 'Women's Store'],
      dtype='object', length=113)

In [98]:
north_york_one_hot["Neighbourhood"]=north_york_venues["Neighbourhood"]
north_york_one_hot.columns

Index(['Airport', 'American Restaurant', 'Arts & Crafts Store',
       'Asian Restaurant', 'Bakery', 'Bank', 'Bar', 'Baseball Field',
       'Basketball Court', 'Beer Store',
       ...
       'Tea Room', 'Thai Restaurant', 'Theater', 'Toy / Game Store', 'Trail',
       'Video Game Store', 'Vietnamese Restaurant', 'Wings Joint',
       'Women's Store', 'Neighbourhood'],
      dtype='object', length=114)

In [99]:
fixed_columns= [north_york_one_hot.columns[-1]]+list(north_york_one_hot.columns[:-1])
fixed_columns


['Neighbourhood',
 'Airport',
 'American Restaurant',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Basketball Court',
 'Beer Store',
 'Bookstore',
 'Boutique',
 'Bubble Tea Shop',
 'Burger Joint',
 'Burrito Place',
 'Bus Line',
 'Bus Stop',
 'Butcher',
 'Café',
 'Candy Store',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Clothing Store',
 'Coffee Shop',
 'Comfort Food Restaurant',
 'Construction & Landscaping',
 'Convenience Store',
 'Cosmetics Shop',
 'Deli / Bodega',
 'Department Store',
 'Dessert Shop',
 'Discount Store',
 'Dog Run',
 'Eastern European Restaurant',
 'Electronics Store',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Food & Drink Shop',
 'Food Court',
 'Food Stand',
 'Fried Chicken Joint',
 'Frozen Yogurt Shop',
 'Furniture / Home Store',
 'Gas Station',
 'Gift Shop',
 'Greek Restaurant',
 'Grocery Store',
 'Gym',
 'Gym / Fitness Center',
 'Halal Restaurant',
 'Hockey Arena',
 'Home Service',

In [100]:
north_york_one_hot = north_york_one_hot[fixed_columns]

In [101]:
north_york_one_hot.head()

,Neighbourhood,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [102]:
north_york_one_hot.shape

(282, 114)

In [103]:
north_york_grouped = north_york_one_hot.groupby("Neighbourhood").mean().reset_index()

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [104]:
north_york_grouped

,Neighbourhood,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Bayview Village,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.25,0.00,0.000000,0.00,0.00
1,"Bedford Park, Lawrence Manor East",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.052632,0.000000,0.052632,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
2,"CFB Toronto, Downsview East",0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
3,Don Mills North,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
4,Downsview Central,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
5,Downsview Northwest,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.045455,0.00,0.00
6,Downsview West,0.00,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.090909,0.00,0.00
7,"Downsview, North Park, Upwood Park",0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.333333,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
8,"Emery, Humberlea",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
9,"Fairview, Henry Farm, Oriole",0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.02,0.000000,...,0.000000,0.020000,0.000000,0.02,0.020000,0.00,0.02,0.000000,0.02,0.06


#### Let's confirm the new size


In [105]:
north_york_grouped.shape

(22, 114)

#### Let's print each neighborhood along with the top 5 most common venues

In [106]:
for i in north_york_grouped["Neighbourhood"]:
    print(F"------>{i}<-------")
    temp = north_york_grouped[north_york_grouped["Neighbourhood"]==i].T.reset_index()
    temp.columns = ["venue","freq"]
    temp = temp.iloc[1:]
    temp["freq"] =temp["freq"].astype(float)
    temp = temp.round({"freq":2})
    top_venues = temp.sort_values("freq",ascending=False).reset_index(drop=True).head(5)
    print(top_venues)
    
    

------>Bayview Village<-------
                        venue  freq
0  Construction & Landscaping  0.25
1                     Dog Run  0.25
2                       Trail  0.25
3                        Park  0.25
4                     Airport  0.00
------>Bedford Park, Lawrence Manor East<-------
                  venue  freq
0    Italian Restaurant  0.11
1           Coffee Shop  0.11
2           Sports Club  0.05
3  Fast Food Restaurant  0.05
4                   Pub  0.05
------>CFB Toronto, Downsview East<-------
         venue  freq
0      Airport  0.25
1  Coffee Shop  0.25
2         Park  0.25
3   Food Court  0.25
4  Men's Store  0.00
------>Don Mills North<-------
          venue  freq
0   Gas Station   0.2
1          Park   0.2
2   Coffee Shop   0.2
3  Soccer Field   0.2
4  Burger Joint   0.2
------>Downsview Central<-------
              venue  freq
0  Insurance Office   0.5
1      Home Service   0.5
2    Massage Studio   0.0
3          Platform   0.0
4       Pizza Place   0.0
---

### Get top 10 venues per neighborhood


In [135]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [151]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = north_york_grouped['Neighbourhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

# neighborhoods_venues_sorted

## 4. Cluster Neighborhoods
#### kmeans to cluster the neighborhoods into 5 clusters

In [152]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

kclusters = 5

north_york_training = north_york_grouped.drop('Neighbourhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_training)

# check cluster labels generated for each row in the dataframe
len(kmeans.labels_)

22

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [153]:
neighborhoods_venues_sorted.shape
# north_york_merged.shape

(22, 11)

In [154]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_york_merged = north_york_df


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood',how="right")

north_york_merged.head() # check the last columns

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.752420,-79.329242,0,Park,Food & Drink Shop,Bus Stop,Women's Store,Food Stand,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant
1,North York,Victoria Village,43.730600,-79.313265,4,Food Stand,Park,Grocery Store,Women's Store,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store
2,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286,4,Clothing Store,American Restaurant,Cosmetics Shop,Furniture / Home Store,Toy / Game Store,Coffee Shop,Electronics Store,Men's Store,Café,Leather Goods Store
3,North York,Don Mills North,43.749055,-79.362227,1,Park,Coffee Shop,Gas Station,Soccer Field,Burger Joint,Women's Store,Food Court,Department Store,Dessert Shop,Discount Store
4,North York,Glencairn,43.707990,-79.448367,4,Pizza Place,Fast Food Restaurant,Asian Restaurant,Grocery Store,Japanese Restaurant,Gas Station,Latin American Restaurant,Mediterranean Restaurant,Sushi Restaurant,Deli / Bodega


Finally, let's visualize the resulting clusters

In [156]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighbourhood'], north_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


#### Cluster 1

In [161]:
north_york_merged.loc[north_york_merged["Cluster Labels"]==0,north_york_merged.columns[[1]+list(range(5,north_york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Food & Drink Shop,Bus Stop,Women's Store,Food Stand,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant


#### Cluster 2

In [162]:
north_york_merged.loc[north_york_merged["Cluster Labels"]==1,north_york_merged.columns[[1]+list(range(5,north_york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Don Mills North,Park,Coffee Shop,Gas Station,Soccer Field,Burger Joint,Women's Store,Food Court,Department Store,Dessert Shop,Discount Store
11,"CFB Toronto, Downsview East",Airport,Coffee Shop,Park,Food Court,Food Stand,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant
14,"Downsview, North Park, Upwood Park",Park,Bakery,Basketball Court,Women's Store,Fried Chicken Joint,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store
19,"Emery, Humberlea",Coffee Shop,Park,Nightclub,Women's Store,Food Court,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant
22,York Mills West,Convenience Store,Speakeasy,Park,Bank,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant
23,Willowdale West,Convenience Store,Bus Line,Intersection,Park,Bakery,Coffee Shop,Eastern European Restaurant,Dessert Shop,Discount Store,Dog Run


#### Cluster 3

In [163]:
north_york_merged.loc[north_york_merged["Cluster Labels"]==2,north_york_merged.columns[[1]+list(range(5,north_york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Hillcrest Village,Residential Building (Apartment / Condo),Dog Run,Women's Store,Food Stand,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Eastern European Restaurant,Electronics Store
10,Bayview Village,Construction & Landscaping,Trail,Park,Dog Run,Food Stand,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Eastern European Restaurant


#### Cluster 4

In [164]:
north_york_merged.loc[north_york_merged["Cluster Labels"]==3,north_york_merged.columns[[1]+list(range(5,north_york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Humber Summit,Construction & Landscaping,Home Service,Food Stand,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store
17,Downsview Central,Insurance Office,Home Service,Women's Store,Fried Chicken Joint,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store


#### Cluster 5

In [165]:
north_york_merged.loc[north_york_merged["Cluster Labels"]==4,north_york_merged.columns[[1]+list(range(5,north_york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Food Stand,Park,Grocery Store,Women's Store,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store
2,"Lawrence Heights, Lawrence Manor",Clothing Store,American Restaurant,Cosmetics Shop,Furniture / Home Store,Toy / Game Store,Coffee Shop,Electronics Store,Men's Store,Café,Leather Goods Store
4,Glencairn,Pizza Place,Fast Food Restaurant,Asian Restaurant,Grocery Store,Japanese Restaurant,Gas Station,Latin American Restaurant,Mediterranean Restaurant,Sushi Restaurant,Deli / Bodega
5,"Flemingdon Park, Don Mills South",Intersection,Coffee Shop,Bubble Tea Shop,Gym,Grocery Store,Beer Store,Supermarket,Women's Store,Electronics Store,Food & Drink Shop
8,"Fairview, Henry Farm, Oriole",Clothing Store,Fast Food Restaurant,Women's Store,Coffee Shop,Japanese Restaurant,Deli / Bodega,Restaurant,Burrito Place,Candy Store,Movie Theater
9,"Northwood Park, York University",Pizza Place,Coffee Shop,Caribbean Restaurant,Massage Studio,Restaurant,Furniture / Home Store,Japanese Restaurant,Fast Food Restaurant,Sushi Restaurant,Bar
13,Downsview West,Grocery Store,Bank,Spa,Clothing Store,Beer Store,Coffee Shop,Fast Food Restaurant,Pizza Place,Hockey Arena,Convenience Store
16,"Newtonbrook, Willowdale",Korean Restaurant,Café,Middle Eastern Restaurant,Pizza Place,Hookah Bar,Grocery Store,Japanese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Dessert Shop
18,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Pharmacy,Café,Butcher,Liquor Store,Fast Food Restaurant,Pub,Juice Bar,Sports Club
20,Willowdale South,Coffee Shop,Ramen Restaurant,Fast Food Restaurant,Café,Indonesian Restaurant,Middle Eastern Restaurant,Plaza,Pizza Place,Pet Store,Movie Theater
